### The Dataset

The data source for this XGBoost model is the UCI ML Repository: https://archive.ics.uci.edu/dataset/551/gas+turbine+co+and+nox+emission+data+set <br>
This project uses the Gas Turbine CO and NOₓ Emission Dataset from the UCI Machine Learning Repository. It contains 36,733 hourly records collected from a gas turbine power plant in Turkey between 2011 and 2015. The dataset includes a mix of ambient and operational variables and can be used to predict carbon monoxide (CO) or nitrogen oxides (NOₓ) emissions.

#### Metadata:
AT - Ambient Temperature (°C)<br>
AP - Ambient Pressure (mbar)<br>
AH - Ambient Humidity (%)<br>
AFDP = Air Filter Differential Pressure (mbar)<br>
GHEP - Gas Turbine Exhaust Pressure (mbar)<br>
TIT - Turbine Inlet Temperature (°C)<br>
TAT- Turbine After Temperature (°C)<br>
TEY - Turbine Energy Yield (MWh)<br>
CDP - Compressor Discharge Pressure (mbar)<br>
CO - Carbon monoxide emissions (mg/m³)<br>
NO - Nitric Oxide emissions (mg/m³)<br>

In [5]:
#Importing necessary libraries for loading the data, feature engineering, and visualizations.
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np   

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")  

/Users/veeksha_work/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/veeksha_work/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [6]:
#reading the csv files
gt11 = pd.read_csv('gt_2011.csv')
gt12 = pd.read_csv('gt_2012.csv')
gt13 = pd.read_csv('gt_2013.csv')
gt14 = pd.read_csv('gt_2014.csv')
gt15 = pd.read_csv('gt_2015.csv')


In [7]:
#how many rows and columns are in the datasets?
print(gt11.shape)
print(gt12.shape)
print(gt13.shape)
print(gt14.shape)
print(gt15.shape)

#show dataframe
gt15.head(3)

(7411, 11)
(7628, 11)
(7152, 11)
(7158, 11)
(7384, 11)


,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,1.95320,1020.1,84.985,2.5304,20.116,1048.7,544.92,116.27,10.799,7.4491,113.250
1,1.21910,1020.1,87.523,2.3937,18.584,1045.5,548.50,109.18,10.347,6.4684,112.020
2,0.94915,1022.2,78.335,2.7789,22.264,1068.8,549.95,125.88,11.256,3.6335,88.147


In [8]:
# Combine dataframes of all years
gt_combined = pd.concat([gt11, gt12, gt13, gt14, gt15], axis=0, ignore_index=True)
gt_combined.shape

(36733, 11)

### Dataset Cleaning

In [9]:
# Checking for missing values
gt_combined.isnull().sum()

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [10]:
# Understanding the data types of the features
for col in gt_combined.columns:
    print(f"{col}: {gt_combined[col].dtype}")

AT: float64
AP: float64
AH: float64
AFDP: float64
GTEP: float64
TIT: float64
TAT: float64
TEY: float64
CDP: float64
CO: float64
NOX: float64


### Feature Engineering

### Model Building 

In [24]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
# Define independent (X) and dependent (y) variables
X = gt_combined.drop('NOX', axis=1)
y = gt_combined['NOX']
X_np = X.values
y_np = y.values

# Split into Training and Testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Manually coded XGBoost regressor

In [12]:
import numpy as np

# Compute gradients and Hessians (second-order)
def compute_grad_hess(y_true, y_pred):
    grad = y_pred - y_true
    hess = np.ones_like(y_true)
    return grad, hess

# Calculate gain from a split with regularization
def calc_split_gain(Gl, Hl, Gr, Hr, reg_lambda, gamma):
    left = Gl ** 2 / (Hl + reg_lambda)
    right = Gr ** 2 / (Hr + reg_lambda)
    parent = (Gl + Gr) ** 2 / (Hl + Hr + reg_lambda)
    gain = 0.5 * (left + right - parent) - gamma
    return gain

# Find best split considering gradients, Hessians & regularization
def best_split(X_col, grad, hess, reg_lambda, gamma, min_child_weight):
    sort_idx = np.argsort(X_col)
    X_sorted = X_col[sort_idx]
    grad_sorted = grad[sort_idx]
    hess_sorted = hess[sort_idx]
    unique_vals = np.unique(X_sorted)
    if len(unique_vals) == 1:
        return None, -np.inf
    thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2
    best_gain, best_thresh = -np.inf, None
    for thr in thresholds:
        left_mask = X_sorted <= thr
        right_mask = ~left_mask
        Gl = grad_sorted[left_mask].sum()
        Hl = hess_sorted[left_mask].sum()
        Gr = grad_sorted[right_mask].sum()
        Hr = hess_sorted[right_mask].sum()
        if Hl < min_child_weight or Hr < min_child_weight:
            continue
        gain = calc_split_gain(Gl, Hl, Gr, Hr, reg_lambda, gamma)
        if gain > best_gain:
            best_gain, best_thresh = gain, thr
    return best_thresh, best_gain

# Fit a tree with second-order info and regularization
def fit_tree_xgb(X, grad, hess, depth, reg_lambda, gamma, min_child_weight):
    if depth == 0 or np.sum(hess) < 2 * min_child_weight:
        G, H = grad.sum(), hess.sum()
        value = -G / (H + reg_lambda)
        return {'leaf': True, 'value': value}
    n_samples, n_features = X.shape
    best_feat, best_thr, best_gain = None, None, -np.inf
    for f in range(n_features):
        thr, gain = best_split(X[:, f], grad, hess, reg_lambda, gamma, min_child_weight)
        if thr is not None and gain > best_gain:
            best_feat, best_thr, best_gain = f, thr, gain
    if best_feat is None:
        G, H = grad.sum(), hess.sum()
        value = -G / (H + reg_lambda)
        return {'leaf': True, 'value': value}
    left_mask = X[:, best_feat] <= best_thr
    right_mask = ~left_mask
    left_tree = fit_tree_xgb(X[left_mask], grad[left_mask], hess[left_mask], depth - 1,
                            reg_lambda, gamma, min_child_weight)
    right_tree = fit_tree_xgb(X[right_mask], grad[right_mask], hess[right_mask], depth - 1,
                             reg_lambda, gamma, min_child_weight)
    return {'leaf': False, 'feature': best_feat, 'threshold': best_thr,
            'left': left_tree, 'right': right_tree}

# Predict with the XGBoost tree (same as before)
def predict_tree_xgb(tree, X):
    preds = np.zeros(X.shape[0])
    for i, x in enumerate(X):
        node = tree
        while not node['leaf']:
            if x[node['feature']] <= node['threshold']:
                node = node['left']
            else:
                node = node['right']
        preds[i] = node['value']
    return preds

In [13]:
# Build XGBoost model incrementally with second-order info and regularization
def build_xgboost(X, y, n_estimators=50, learning_rate=0.1,
                  max_depth=3, reg_lambda=1.0, gamma=0.0, min_child_weight=1):
    X, y = np.asarray(X), np.asarray(y)
    y_pred = np.full(len(y), y.mean())
    trees = []
    for _ in range(n_estimators):
        grad, hess = compute_grad_hess(y, y_pred)
        tree = fit_tree_xgb(X, grad, hess, max_depth, reg_lambda, gamma, min_child_weight)
        update = predict_tree_xgb(tree, X)
        y_pred += learning_rate * update
        trees.append(tree)
    return {'init_value': y.mean(), 'trees': trees, 'learning_rate': learning_rate}


In [18]:
# Training
model = build_xgboost(X_train, y_train, n_estimators=100, learning_rate=0.1,
                      max_depth=3, reg_lambda=1.0, gamma=0.1, min_child_weight=5)

####  XGBoost regressor built using xgboost library

In [25]:
# Base model
xgb = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42)

# Step 4: Train the Model
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

### Predictions

####  Manually coded XGBoost regression predictions

In [26]:
# Predict with manual XGBoost model
def predict_xgboost(X, model):
    X = np.asarray(X)
    y_pred = np.full(X.shape[0], model['init_value'])
    for tree in model['trees']:
        y_pred += model['learning_rate'] * predict_tree_xgb(tree, X)
    return y_pred

In [27]:
y_pred_manual = predict_xgboost(X_test, model)

#### xgboost library built XGBoost regression predictions

In [29]:
# Step 5: Predict with library XGBoost model
y_pred = xgb.predict(X_test)

### Model Evaluations

#### Evaluating manual xgboost model

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate
mae = mean_absolute_error(y_test, y_pred_manual)
mse = mean_squared_error(y_test, y_pred_manual)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_manual)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

MAE: 4.19
MSE: 34.27
RMSE: 5.85
R² Score: 0.7414


#### Evaluating xgboost library model

In [31]:
# Evaluate
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

MAE: 2.43
MSE: 12.89
RMSE: 3.59
R² Score: 0.9027


In [32]:
# Printing CLV data stats for model eval metric comparison
print(y.max())
print(y.mean())
print(y.min())

119.91
65.29306726921297
25.905
